# Repaso: Manejo de Pandas

## Selección de datos

Antes de nada vamos a recordar cómo seleccionar atributos e instancias.

Cargamos pandas.

In [ ]:
from urllib.request import ProxyBasicAuthHandler
import pandas as pd
from pandas.core.common import random_state

Leemos datos

In [ ]:
df = pd.read_csv("sw_characters.csv")
print(df.head(2))

Podemos consultar los atributos con:

In [ ]:
df.columns

## Selección de atributos

Sobre este conjunto de datos haremos las siguientes operaciones de
selección (en todas ellas el resultado es un nuevo conjunto de datos):

- selección de atributos concretos.
- selección de instancias concretas.


1. selección de 3 variables en concreto: **name**, **height**, **gender**.

In [ ]:
# se seleccionan algunas caracteristicas: name, height
# and gender
data1 = df[["name", "height", "gender"]]
print(data1.head(3))

2. todas las variables excepto las indicadas a continuación:
**birth_year** y **gender**.


In [ ]:
# se mantienen todas las variables exceptuando
# birth_year y gender
df2 = df.drop(['birth_year', 'gender'], axis=1)
df2.columns

## Selección de instancias

Es fácil filtrar un valor numérico o por valor exacto:

In [ ]:
data1[data1["gender"].isnull()]

Se combina con & y | (no dobles) usando paréntesis:

In [ ]:
selected = (data1["gender"].isnull()) & (data1["height"] < 100)
data1[selected]

Es más difícil si queremos filtrar según uno o varios valores:

In [ ]:
df2.skin_color.describe()

Por varios valores

In [ ]:
df2[df2.skin_color.isin(["blonde", "brown"])]

Expresiones regulares

In [ ]:
df2[(~df2.skin_color.isnull()) &
    (df2.skin_color.str.contains("blo*", case=True,regex=True))]

## Renombrado de variables

Se puede hacer directamente editando columns:

In [ ]:
df_tmp = df2.copy()
df_tmp.columns = ["V1","V2", "V3", "V4", "V5", "V6", "V7", "V8"]
df_tmp.columns

Pero lo suyo es renombrar usando un diccionario:

In [ ]:
df2.rename(columns={"skin_color": "color_piel"}, inplace=True)
df2.columns

## Conocer los tipos originales del dataset

`info` nos devuelve los tipos (`object` son `string`)

In [ ]:
df.info()

Para conocer información sobre los valores numéricos se puede hacer:

In [ ]:
df.describe()

Y más en detalle se puede usar `describe` con un atributo:

In [ ]:
df.species.describe()

Para ver las frecuencias se puede usar `values_counts()`.

In [ ]:
df.species.value_counts().head()

Se puede normalizar (y no ordenar si se quiere):

In [ ]:
df.species.value_counts(normalize=True).head()

`value_counts()` también permite medir frecuencia de combinaciones:

In [ ]:
df[["species", "hair_color"]].value_counts(normalize=True).head(8)

In [ ]:
df[["species", "hair_color"]].value_counts(normalize=False, ascending=True).head()

# Valores perdidos

## Valores perdidos

Un problema habitual suele consistir en la presencia de datos datos.

Es importante tener claro cómo leer los datos indicando la posible ausencia de
valor, usando *na_values*:

In [ ]:
data = pd.read_csv("nulos.csv", na_values=['?', '', 'NA'])
data.isnull().sum()

Hay múltiples técnicas para tratar los datos perdidos. Es importante valorar si la
técnica de aprendizaje es capaz de trabajar con datos perdidos o no.

Para conocer los nulos (en porcentaje):

In [ ]:
ratio_nulos = data.isnull().sum()/data.shape[0]
ratio_nulos

## Opción directa (eliminar nulos)

Se pueden eliminar o bien los atributos que tienen demasiados nulos, o eliminar
tuplas.

Eliminar atributos que superen un umbral:

In [ ]:
data2 = data.copy()

for i, atrib in enumerate(data):
    if ratio_nulos[i] > 0.4:
        data2.drop(atrib, axis=1, inplace=True)

print(data2)

Eliminar todas las filas con algún nulo

In [ ]:
data_drop = data.dropna()
print(data_drop)

In [ ]:
data_drop = data2.dropna()
print(data_drop)

## Tratar valores perdidos con paquetes externos

Dado que el aprendizaje en `scikit-learn` no es compatible con valores perdidos,
vamos a probar distintas opciones que la propia librería nos permite.

In [ ]:
#| echo: false
import pandas as pd
import numpy as np
from sklearn import datasets
iris_dataset = datasets.load_iris(as_frame=True)
X_iris = iris_dataset.data.copy()

Para probar los métodos añadidos nulos al dataset:

In [ ]:
#Prepare the dataset to test sk-learn imputation values tools
np.random.seed(42)
rows = np.random.randint(0, np.shape(X_iris)[0], 50)
# No modifico la última característica
cols = np.random.randint(0, np.shape(X_iris)[1]-1, 50)
X_iris_missing = X_iris.to_numpy()
#Add missing values in random entries from the iris dataset
X_iris_missing[rows, cols] = np.NaN
X_iris_missing = pd.DataFrame(X_iris_missing, columns=X_iris.columns)

Tenemos ahora nulos

In [ ]:
print(X_iris_missing.iloc[:10,:])

## Imputación Univariante

Los objetos de tipo *Impute* permite reemplazar los valores nulos. Para ello
pueden usar un valor constante o una estadística (media, mediana o más
frecuente) para cada columna con nulos.


In [ ]:
from sklearn.impute import SimpleImputer
# strategy puede ser "mean", "median", "most_frequent", "constant".
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_X = pd.DataFrame(imp.fit_transform(X_iris_missing), columns=X_iris.columns)
print(imputed_X.iloc[:10,:])

## Usando KNN

Se pueden imputar usando el algoritmo de K vecinos (KNN). Para cada atributo
perdido se calcula a partir de los K vecinos más cercanos que no sea nulo. Los
vecinos pueden ser diferentes para cada atributo.

Si no encuentra vecinos sin nulos, el atributo es borrado.

In [ ]:
from sklearn.impute import KNNImputer
Knn_imp = KNNImputer(n_neighbors=4).fit(X_iris_missing)
imputed_X = pd.DataFrame(Knn_imp.transform(X_iris_missing), columns=X_iris.columns)
print(imputed_X.iloc[:10,:])

# Normalización de entrada

## Estandarización

Estandarización es un requisito de muchos modelos de ML, como los basados en
distancias.

`scikit-learn` permite hacer estandarización, hay [múltiples
opciones](]https://scikit-learn.org/stable/modules/preprocessing.html)

In [ ]:
#Build a preprocessing object
from sklearn.preprocessing import StandardScaler
iris_dataset = datasets.load_iris(as_frame=True)
X_iris = iris_dataset.data.copy()
scaler = StandardScaler().fit(X_iris)
#Check the mean and the std of the training set
print(scaler.mean_)
print(scaler.scale_)

Una vez entrenado se puede aplicar:

In [ ]:
X_iris_scaled = scaler.transform(X_iris)
print(X_iris.iloc[:5,:])
print("StandardScaler: ")
print(X_iris_scaled[:5,:])

Confirmemos:

In [ ]:
#Transform the dataset using the preprocessin object and check results
X_scaled = pd.DataFrame(scaler.fit_transform(X_iris), columns=X_iris.columns)
print(X_scaled.mean(axis=0))
print(X_scaled.std(axis=0))

Visualmente

In [ ]:
from matplotlib import pyplot as plt
fig,axs = plt.subplots(2,1)
X_iris.plot.kde(ax=axs[0])
X_scaled.plot.kde(ax=axs[1])

Otro muy común es `MinMaxScaler`:

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_iris_scaled2 = MinMaxScaler().fit_transform(X_iris)
print(X_iris.iloc[:5,:])
print("MinMaxScaler: ")
print(X_iris_scaled2[:5,:])

## Normalización

La normalización es escalar las muestras individuales para que tenga una normal
unidad.

Es esencial para espresiones cuadráticas, o que usen un *kernel* que mida similaridad de
pares de instancias.

In [ ]:
from sklearn.preprocessing import normalize
print(X_iris.iloc[:4,:])
X_normalized = pd.DataFrame(normalize(X_iris), columns=X_iris.columns)
print(X_normalized.iloc[:4,:])

# Atributos categóricos

## Codificando atributos categóricos

Es común atributos con valores categóricos. `Scikit-learn` no es capaz de
procesarlos, por lo que es necesario transformarlo a valores numéricos.

- LabelEncoder y OrdinalEncoder: Asigna un valor numérico por cada categoría.

- OneHotEncoder: Codifica cada categoría usando una nueva columna.


Los Label/OrdinalEncoder asigna un orden entre las categorías que suele ser
'falso' si ese concepto no existe.

## LabelEncoder y OrdinalEncoder

Ambos asignan un valor numérico distinto a cada categoría.

Diferencia:

- `OrdinalEncoder` puede procesar varias columnas, se usa para características.

- `LabelEncoder` solo procesa un elemento, se usa para el atributo objetivo
  (*target*).


Advertencia: Evitar hacer esto:

In [ ]:
from sklearn.preprocessing import LabelEncoder
targets_train = ["rubio", "moreno", "pelirrojo", "azul"]
targets_test = ["moreno", "pelirrojo"]

targets_train_num = LabelEncoder().fit_transform(targets_train)
targets_test_num = LabelEncoder().fit_transform(targets_test)
print(targets_train)
print(targets_train_num)
print(targets_test)
print(targets_test_num)

Las etiquetas no coinciden.


Para evitarlo hay que hacer *fit* solo con el de entrenamiento.

In [ ]:
labeler_target = LabelEncoder()
targets_train_num = labeler_target.fit_transform(targets_train)
targets_test_num = labeler_target.transform(targets_test)
print(targets_train)
print(targets_train_num)
print(targets_test)
print(targets_test_num)

Guardar siempre los labeler (diccionario por nombre, ...).


Ejemplo:

In [ ]:
data_train_df = pd.DataFrame({'age': [30, 41, 42, 21],
                         'pelo': targets_train,
                         'ojos': ['azules', 'verdes', 'marrones', 'marrones']})
data_test_df = pd.DataFrame({'age': [25, 23],
                             'pelo': targets_test,
                             'ojos': ['verdes', 'azules']})
print(data_train_df)

Vamos a aplicar el etiquetado.


- Opción 1: Sólo con Label Encoder:

In [ ]:
labelers = {}
cols = {}
atribs = ["pelo", "ojos"]
data_train_num = data_train_df.copy()
data_test_num = data_test_df.copy()

for i in atribs:
    cols[i] = LabelEncoder()
    data_train_num[i] = cols[i].fit_transform(data_train_num[i])
    data_test_num[i] = cols[i].transform(data_test_num[i])

print(data_train_num)
print(data_test_num)

- Opción 2: Usando `OrdinalEncoder`

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
atribs = ["pelo", "ojos"]
labelers = OrdinalEncoder(dtype=np.int32) # Por defecto usa float
data_train_num = data_train_df.copy()
data_test_num = data_test_df.copy()

data_train_num[atribs] = labelers.fit_transform(data_train_df[atribs])
data_test_num[atribs] = labelers.transform(data_test_df[atribs])

print(data_train_num)
print(data_test_num)

## *LabelEncoder*, *OrdinalEncoder* y orden

Esta codificación está considerando un orden entre categorías.

En algunos casos como ['pequeño', 'mediano', 'grande'] puede tener sentido pero
la mayoría de las veces no.

Cuando no (como 'pelo' o 'color' del ejemplo anterior) es necesario aplicar
*OneHotEncoder*.


`OneHotEncoder` crea una columna por categoría (ej: 'azul') indicando si se cumple
o no.

- Aumenta el número de columnas.

- Evita suponer un orden.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Por defecto es matriz sparse
encoder = OneHotEncoder(sparse=False, dtype=np.int32)
data_train_hot = encoder.fit_transform(data_train_df[atribs])
print(data_train_hot)

Se puede convertir a *dataframe*:

In [ ]:
new_columns = encoder.get_feature_names_out()
print(new_columns)
data_train_hot = pd.DataFrame(data_train_hot, columns=new_columns)
# Copio el resto de atributos
data_train_hot['age'] = data_train_df['age']
print(data_train_hot)

## Dummies en Pandas

Pandas ya soporta el *hotencoding*, pero presenta problemas.

In [ ]:
pd.get_dummies(data_train_df[['pelo', 'ojos']])

Recomiendo usar `OneHotEncoder` por tener más opciones.

## Valores binarios

Si el valor numérico es binario, ej: vivo/muerto no es necesario aplicar el
*hotencoding*.

In [ ]:
columns = ["Employed", "Place", 'Browser']
X = [['employed', 'from US', 'uses Safari'], ['unemployed', 'from Europe', 'uses Firefox'], ['unemployed', 'from Asia', 'uses Chrome']]
enc = OneHotEncoder(drop='if_binary')
trans_X = enc.fit_transform(X)
transformed_X = pd.DataFrame(trans_X.toarray(), columns=enc.get_feature_names_out())
print(transformed_X)

# Pipelines y ColumnTransformer

## Aplicar `OneHotEncoder`

Scikit-learn permite combinar transformaciones con `ColumnTransformer`.

In [ ]:
from sklearn.compose import make_column_transformer

transformer = make_column_transformer(
    (OneHotEncoder(), ['pelo', 'ojos']),
  remainder='passthrough') # Para ignorar el resto y no dar error

transformed = transformer.fit_transform(data_train_df)
data_train_num = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
print(data_train_num)

`ColumnTransformer` permite procesar distintos datasets.

En conjunción con make_column_selector (que permite filtrar atributos por su tipo) es muy potente y cómodo.

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

X = pd.DataFrame({'city': ['London', 'London', 'Paris', 'Sallisaw'],
                  'rating': [5, 3, 4, 5]})
ct = make_column_transformer(
      (StandardScaler(),
       make_column_selector(dtype_include=np.number)),  # rating
      (OneHotEncoder(),
       make_column_selector(dtype_include=object)))  # city
ct.fit_transform(X)

## Pipelines

Facilitan aplicar distintos preprocesamientos.

Un *pipeline* se compone de una serie de transformaciones que van sufriendo el
dataset (se puede incluir el modelo a aprender).

Un *pipeline* se usa igual que un modelo.

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
iris_targets = iris_dataset.target
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_iris, iris_targets,
                                                    random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

También se puede aplicar con validación cruzada:

In [ ]:
from sklearn.model_selection import cross_val_score

pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
scores = cross_val_score(pipe, X_iris, iris_targets, cv=5)
print(scores)
print(scores.mean())

Por comodidad se puede usar `make_pipeline` con tantos atributos como
procesamientos y/o mdelos.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

model = make_pipeline(StandardScaler(), SVC(C=1))
cross_val_score(model, X_iris, iris_targets, cv=5)

## Pipelines y ColumnTransformer

También se pueden combinar con `ColumnTransformer`.

In [ ]:
from sklearn.model_selection import cross_val_score
trans = make_column_transformer(
    (StandardScaler(), ["age"]),
    (OneHotEncoder(), ["pelo", "ojos"])
    )
trans.fit_transform(data_train_df)
pipe = make_pipeline(trans, SVC())
print(data_train_df)
pipe.fit(data_train_df, [0, 0, 1, 1])
print(data_test_df)
pipe.predict(data_test_df)

# Discretización

## Discretización usando rangos

A menudo no nos interesa un valor numéricos (ej: `age`) sino convertirlo en un conjunto discreto de valores (*joven*, *adulto*, *mayor*).

La clase `K-bins` permite discretizar.

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
#Build a discretizer object indicating three bins for every feature
est = KBinsDiscretizer(n_bins=[3, 3, 3, 3], encode='ordinal').fit(X_iris)
#Check feature maximum and minimum values 
# print(np.max(X_iris, axis = 0))
# print(np.min(X_iris, axis = 0))
#Check binning intervals
print(est.bin_edges_)

In [ ]:
#Print discretization results
print(X_iris.iloc[:5,])
discretized_X = pd.DataFrame(est.transform(X_iris), columns=X_iris.columns)
print(discretized_X.iloc[:5,])

## Distintas estrategias de discretización

El criterio de discretización puede ser cambiado con el parámetro `strategy`.

Una tendencia común sería una uniforme:


In [ ]:
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
age_disc = est.fit_transform(data_train_df[['age']])
print(est.bin_edges_)
print(age_disc)

No todos los rangos tienen interés, pueden concentrarse.


A menudo la mejor estrategia depende de la frecuencia (comportamiento por defecto).

In [ ]:
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
age_disc = est.fit_transform(data_train_df[['age']])
print(est.bin_edges_)
print(age_disc)

De esta manera, discretiza más en detalle los intervalos más comunes.


La otra opción es la estrategia `kmean` que aplica una clasificación `kmeans` sobre cada algoritmo.

In [ ]:
#Build a discretizer object indicating three bins for every feature and using the kmeans strategy
est = KBinsDiscretizer(n_bins=[3, 3, 3, 3], encode='ordinal', strategy='kmeans').fit(X_iris)
#Check binning intervals and results
print(est.bin_edges_)
discretized_X = pd.DataFrame(est.transform(X_iris), columns=X_iris.columns)
print(discretized_X.iloc[:5,])

# Reducción de dimensionalidad

## Reducción de dimensionalidad

Una de las prácticas más comunes de procesamiento de datos es la reducción de la
dimensional, lo cual ayuda a transformar o seleccionar las características que mejor
representan la estructura, y que por tanto, son más adecuadas para el
aprendizaje.

## Reducción de la Dimensionalidad No supervisada

Si el número de características es alto, puede ser útil `reducirlas` mediante
una fase no supervisada.

### Principal Components Analysis **(PCA)**

Descomponer un *dataset* multivariante en un conjunto de componentes ortogonales
que explica la cantidad de varianza.

In [ ]:
from sklearn import preprocessing, datasets, decomposition

#Load the iris dataset and scale it
X_iris, y_iris = datasets.load_iris(return_X_y=True)

scaler = preprocessing.StandardScaler().fit(X_iris)
X_scaled = scaler.transform(X_iris)

Aplico el PCA:

In [ ]:
#Apply principal componentes analysis to reduce the iris number of features from 4 to 3
pca = decomposition.PCA(n_components=3)
X_reduced = pca.fit_transform(X_scaled)
print(X_scaled[:3,:])
print("Con menos dimensión")
print(X_reduced[:3,:])

Visualizando en 2D el PCA:

In [ ]:
import seaborn as sns
df = pd.DataFrame(X_reduced, columns=["X1", "X2", "X3"])
#Plot the iris dataset in a 2D pairplot
df["target"] = iris_dataset.frame["target"]
g = sns.pairplot(df, hue="target", palette=sns.color_palette("hls", 3),
                 height=1.25, aspect=2)
plt.show()

# Selección de Características

## Selección de Características

En este caso se reducen las características eligiendo las características que
permitirían un mejor desempeño del clasificador.

## Selección de Características Secuencial

La selección secuencial (*Forward-SFS*) busca iterativamente una nueva
característica a añadir a las ya seleccionadas. Empieza con cero características
y escoge aquella que maximiza aplicando CV usando un estimador (cualquiera le
vale, pero mejor que no sea lento) sobre una única característica.

Luego repite el procedimiento añadiendo una nueva característica cada vez, hasta
terminar con el número pedido de características.

## Backward-SFS

*Backward-SFS* sigue la misma idea, pero al revés, en vez de ir añadiendo va
eliminando características aplicando un estimador.

No dan los mismos resultado, ni son igualmente eficientes. Si tenemos 10
características y queremos siete será más eficiente *Backward-SFS* que
*Forward-SFS*.

`Scikit-learn` ofrece `SequentialFeatureSelector` que implementa ambos (
direction puede ser *forward* o *backward*).


Ejemplo:

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import ExtraTreesClassifier
import warnings
warnings.filterwarnings('ignore')

diabetes = datasets.load_diabetes()
X_diabetes = diabetes.data
y_diabetes = diabetes.target

clf = ExtraTreesClassifier(n_estimators=50)
#Perform FORDWARD feature selection over the diabetes dataset to reduce it to 3 dimensions
sfs_forward = SequentialFeatureSelector(clf, n_features_to_select=3, direction="forward")
sfs_forward_fitted = sfs_forward.fit(X_diabetes, y_diabetes)

X_reduced_for = sfs_forward.transform(X_diabetes)
print(X_reduced_for.shape)
print(sfs_forward_fitted.get_support())
atribs = np.array(diabetes.feature_names)
print("Atributos elegidos")
print(atribs[sfs_forward_fitted.get_support()])

## Aplicando con *backward*

In [ ]:
#Perform BACKWARD feature selection over the diabetes dataset to reduce it to 3 dimensions
sfs_backward = SequentialFeatureSelector(clf, n_features_to_select=3, direction="backward")
sfs_backward_fitted = sfs_backward.fit(X_diabetes, y_diabetes)

X_reduced_back = sfs_backward_fitted.transform(X_diabetes)
print(X_reduced_back.shape)
print(sfs_backward_fitted.get_support())
print("Atributos elegidos")
print(atribs[sfs_backward_fitted.get_support()])

# Datos anómalos

## Datos anómalos

A veces en los datos se presentan valores anómalos, que se han introducido, por
ejemplo, debido a errores en los procesos de recogida de datos.

Quizás el valor anómalo se deba a una cambio en la distribución de valores y no
a un error.

La intuición básica en las técnicas detección de anomalías es:

- La mayoría de los datos siguen una determinada distribución.
- Los datos las anomalías representan entonces una distribución distinta,
que no coincide con el resto.

## Detección *a mano*

Un método clásico es considerar como datos anómalos aquellos para los que el
valor de un atributo esté fuera del 1.5*rango intercuartil

Supongamos unos datos:

In [ ]:
# Generate train data
rng = np.random.RandomState(42)
X_orig = 0.3 * rng.randn(100, 2)
#X_good = X_orig-4
X_good = np.r_[X_orig + 2, X_orig - 2]
# Generate some abnormal novel observations
X_outliers = rng.uniform(low=5, high=8, size=(10, 2))
X = np.vstack([X_good, X_outliers])
np.random.shuffle(X)
X_df = pd.DataFrame(X, columns=["V1", "V2"])
print(X_df.shape)
print(X_df.columns)

In [ ]:
sns.catplot(X_df)
plt.show()

In [ ]:
sns.catplot(X_df, kind="box")
plt.show()

Vamos a aplicar:

In [ ]:
# eliminar outliers como aquellos casos fuera de 1.25 veces el rango intercuartil
ratio = 1.25
Q1 = X_df.quantile(0.25)
Q3 = X_df.quantile(0.75)
IQR = Q3 - Q1
outliers = ((X_df < (Q1 - ratio * IQR)) |(X_df > (Q3 + ratio * IQR))).any(axis=1)
X_df_irq= X_df.copy()
X_df_irq["outlier"] = outliers

Visualizamos:

In [ ]:
sns.relplot(x="V1", y="V2", data=X_df_irq, hue="outlier", aspect=2)
plt.show()

## Detección Outliers usando scikit-learn (*LocalOutlierFactor*)

Esta técnica mide la desviación local de una muestra respecto a los vecinos
(usando k-vecinos). Al comparar la distancia local con la de los vecinos, se
ientifica las instancias con una densidad sustanciamente menor que sus vecinos.

Cerca de -1 si lo considera *outlier*, 1 en caso contrario.

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=10)
X_df_loc = X_df.copy()
X_df_loc["outlier"] = np.abs(clf.fit_predict(X_df) - -1) <= 1e-3
print(X_df_loc.head(3))

Visualizamos:

In [ ]:
sns.relplot(x="V1", y="V2", data=X_df_loc, hue="outlier", aspect=2)
plt.show()